In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sales = pd.read_csv(r'C:\Users\Chaim\Desktop\Master M2\Smart_Sales_Dashboard\Smart_Sales_Dashboard\data\clean\sales_clean.csv')
customers = pd.read_csv(r'C:\Users\Chaim\Desktop\Master M2\Smart_Sales_Dashboard\Smart_Sales_Dashboard\data\clean\customers_clean.csv')
customers.head
# Convert dates
sales['invoice_date'] = pd.to_datetime(sales['invoice_date'], errors='coerce')


In [8]:
# Reference date
max_date = sales['invoice_date'].max()

rfm = sales.groupby('customer_id').agg({
    'invoice_date': lambda x: (max_date - x.max()).days,   # Recency
    'invoice_no': 'nunique',                               # Frequency
    'total': 'sum'                                         # Monetary
}).reset_index()

rfm.columns = ['customer_id', 'recency', 'frequency', 'monetary']


Customer Segmentation 

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Drop NaNs from RFM
rfm_clean = rfm[['recency', 'frequency', 'monetary']].dropna()

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(rfm_clean)

# Determine inertia for elbow method
inertia = []
K_range = range(1, 8)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

# Fit KMeans with chosen number of clusters
kmeans = KMeans(n_clusters=4, random_state=42).fit(X)

# Assign cluster labels back to rfm_clean
rfm_clean['cluster'] = kmeans.labels_

# Save the clustered dataset
rfm_clean.to_csv(r'C:\Users\Chaim\Desktop\Master M2\Smart_Sales_Dashboard\Smart_Sales_Dashboard\data\customers_clusters.csv', index=False)
